In [1]:
!pip install moviepy
import numpy as np
import librosa
from moviepy.editor import VideoFileClip
import keras
import pandas as pd
# Define the input video file and output audio file
mp4_file = "video.mp4"
mp3_file = "sample3.wav"

# Load the video clip
video_clip = VideoFileClip(mp4_file)

# Extract the audio from the video clip
audio_clip = video_clip.audio

# Write the audio to a separate file
audio_clip.write_audiofile(mp3_file)

# Close the video and audio clips
audio_clip.close()
video_clip.close()


MoviePy - Writing audio in sample3.wav


MoviePy - Done.


In [2]:
def extract_features(data):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # without augmentation
    res1 = extract_features(data)
    result = np.array(res1)
    
    # data with noise
    res2 = extract_features(data)
    result = np.vstack((result, res2)) # stacking vertically
    
    # data with stretching and pitching
    res3 = extract_features(data)
    result = np.vstack((result, res3)) # stacking vertically
    
    return result

In [3]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [10]:
def identify_aggressive(audio_path):
    X = []
    feature = get_features(audio_path)
    for ele in feature:
        X.append(ele)  
    Features = pd.DataFrame(X)
    X = Features.iloc[: ,:-1].values
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = np.expand_dims(X, axis=2)
    pred = reconstructed_model.predict(X)
    if "angry" in pred:
        print("Agressive behavior identified. Please review footage: ", audio_path)

In [11]:
sample_rate = 44100
reconstructed_model = keras.models.load_model("audio_model_weights.keras")
identify_aggressive(mp3_file)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
